# Multiple News Phase diagrams

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import random

ModuleNotFoundError: No module named 'pandas'

In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), 'classes'))

In [5]:
from agent import AgentState
from news import News
from utils import construct_world
from simulation import Simulation
from visualization import Visualization

In [6]:
def create_news(sensations, decay_parameters):
    news = {}
    for name, (s, c) in enumerate(zip(sensations, decay_parameters)):
        news[name] = News(name, sensation=s, decay_parameter=c)
        
    return news

def create_sensation_map(sensations_1, sensations_2, num_agents=100):
    agent_names = list(range(num_agents))
    thresholds = np.random.normal(0.6, 0.1, num_agents)
    independences = np.random.normal(0.4, 0.1, num_agents)
    
    df = pd.DataFrame(columns=['sensation news 1', 'sensation news 2', 'number active'])
    
    print(f'Starting simulation with {num_agents} agents (this may take some time...)')  
    for s_1 in sensations_1:
        for s_2 in sensations_2:
            news = create_news([s_1, s_2], [0.0, 0.0])
            world = construct_world(agent_names, thresholds, independences, news)
            
            initial_agent_names = np.random.choice(agent_names, len(news.keys()))
            for idx, name_news in enumerate(news.keys()):
                world.agents[initial_agent_names[idx]].states[name_news] = AgentState.ACTIVE
                
            number_active, _, _ = world.full_dynamics()
            cascade_size = (number_active[0] - number_active[1]) / num_agents
            
            df_active = pd.DataFrame([[s_1, s_2, cascade_size]], columns=df.columns)
            df = df.append(df_active, ignore_index=True)
            
    print('Simulation completed!!!')
    return df.astype(float).pivot('sensation news 2', 'sensation news 1', 'number active')

In [9]:
sensation_1 = np.linspace(0.0, 1.0, 11)
sensation_2 = np.linspace(0.0, 1.0, 11)

sensation_map = create_sensation_map(sensation_1, sensation_2)

Starting simulation with 100 agents (this may take some time...)
Simulation completed!!!


In [7]:
ax = sns.heatmap(sensation_map, cmap='YlGnBu', vmin=-1.0, vmax=1.0)
ax.invert_yaxis()

NameError: name 'sns' is not defined